In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
from pororo import Pororo
qg = Pororo(task = 'qg', lang = 'ko')

[Korean Sentence Splitter]: Initializing Kss...


In [3]:
wiki_df = pd.read_json('/opt/ml/data/wikipedia_documents.json', orient = 'index')
print(len(wiki_df['text'].unique()))

56737


In [4]:
texts = []
titles = []

for i in range(len(wiki_df)) :
    wiki_context = wiki_df['text'][i]
    wiki_title = wiki_df['title'][i]

    if wiki_title in wiki_context :
        texts.append(wiki_context)
        titles.append(wiki_title)

wiki_qa_df = pd.DataFrame(data = {'text':texts, 'title':titles})

## General with Similarity & exclude

In [12]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(
    'klue/bert-base',
    use_fast=True,
)
model = AutoModel.from_pretrained('klue/bert-base')

In [7]:
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [10]:
sample = wiki_qa_df.sample(100)
sample.reset_index(drop = True, inplace = True)

In [11]:
tmp_generation_data = {'question': [],
                   'context': [],
                   'title': [],
                   'id': [],
                   'answers': [],
                   'document_id': [],
                   '__index_level_0__': []}
# __index_level_0__: 10001부터 번호 Numbering\n",
# document_id: 100001부터 Numbering\n",
# answers: 알맞게 삽입\n",
# id: Aug-0-00001부터 Nubering\n",
error_list = []
for i in tqdm(range(len(sample))) :
    text = wiki_qa_df['text'][i]
    title = wiki_qa_df['title'][i]
    try :
        question = qg(title, text)
    except :
        error_list.append(i)
        continue
    answer_start = text.index(title)
    answers = {'answer_start': [answer_start], 'text': [title]}
    
    query_cls_hidden = get_cls_token(question)
    passage_cls_hidden = get_cls_token(text)
    similarity = cosine_similarity(query_cls_hidden, passage_cls_hidden)

    if similarity[0][0] >= 0.6 and title not in question :
        __index_level_0__ = 10001 + i
        document_id = 100001 + i
        id_number = 1 + i
        id_number = str(id_number)
        id_ = f'Aug-0-{id_number.zfill(5)}'
        
        tmp_generation_data['title'].append(title)
        tmp_generation_data['context'].append(text)
        tmp_generation_data['question'].append(question)
        tmp_generation_data['id'].append(id_)
        tmp_generation_data['answers'].append(answers)
        tmp_generation_data['document_id'].append(document_id)
        tmp_generation_data['__index_level_0__'].append(__index_level_0__)


 31%|███       | 31/100 [00:30<01:03,  1.09it/s][nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
 79%|███████▉  | 79/100 [01:08<00:16,  1.29it/s][nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
100%|██████████| 100/100 [01:23<00:00,  1.19it/s]


In [14]:
tmp_generation_data = pd.DataFrame(tmp_generation_data)

In [15]:
tmp_generation_data

,question,context,title,id,answers,document_id,__index_level_0__
0,가나에 올림표기를 한 문자는?,일본어 표기에 많이 쓰이는 올림문자 (후리가나)는 그대로 올려쓰지 않고 '｜'나 '...,아오조라 문고,Aug-0-00006,"{'answer_start': [174], 'text': ['아오조라 문고']}",100006,10006
1,수소와 헬륨의 위치에 대한 논쟁이 이어지고 있는 증거는?,수소와 헬륨의 위치에 대한 논쟁이 이어지고 있다. 현재의 주기율표에서는 수소를 알칼...,주기율표,Aug-0-00009,"{'answer_start': [32], 'text': ['주기율표']}",100009,10009
2,생물의 몸을 구성하는 단백질의 기본 구성 단위를 뭐라고 해?,"아미노산(amino acid)은 생물의 몸을 구성하는 단백질의 기본 구성 단위로, ...",아미노산,Aug-0-00010,"{'answer_start': [0], 'text': ['아미노산']}",100010,10010
3,일본어에서 쓰는 두 가지 가나가 뭐야?,"히라가나(平仮名, ひらがな, Hiragana)는 일본어에서 사용하는 두 가지 가나 ...",히라가나,Aug-0-00012,"{'answer_start': [0], 'text': ['히라가나']}",100012,10012
4,"관찰 및 조사로 얻을 수 있는 데이터로부터, 응용 수학의 기법을 이용해 수치상의 성...","통계학은 관찰 및 조사로 얻을 수 있는 데이터로부터, 응용 수학의 기법을 이용해 수...",통계학,Aug-0-00015,"{'answer_start': [0], 'text': ['통계학']}",100015,10015
5,다양한 분야의 연구에서 주어진 문제에 대한 해답을 구하는 방법을 연구하는 과학의 한...,매우 다양한 분야의 연구에서 주어진 문제에 대하여 적절한 정보를 수집하고 분석하여 ...,통계학,Aug-0-00017,"{'answer_start': [89], 'text': ['통계학']}",100017,10017
6,기원전 2~1세기 그리스의 히파르코스와 프톨레마이오스 등은 각도에 대해 달라지는 현...,기원전 2~1세기 그리스의 히파르코스와 프톨레마이오스 등은 각도에 대해 달라지는 현...,삼각함수,Aug-0-00021,"{'answer_start': [75], 'text': ['삼각함수']}",100021,10021
7,자연계의 기본 입자와 중력을 제외한 그 상호작용을 다루는 게이지 이론은?,"소립자 물리학의 표준 모형(標準模型, Standard Model)은 자연계의 기본 ...",표준 모형,Aug-0-00022,"{'answer_start': [9], 'text': ['표준 모형']}",100022,10022
8,이론적으로 여러 자연스러움 문제를 안고 있는 모형은?,표준 모형은 이론적으로 여러 자연스러움 (naturality) 문제를 안고 있다. ...,표준 모형,Aug-0-00023,"{'answer_start': [0], 'text': ['표준 모형']}",100023,10023
9,입자 물리학의 실험 결과를 오차 범위 안에 설명하기 위해 만들어진 모형이 뭐야?,표준 모형은 입자 물리학의 거의 모든 실험 결과를 오차 범위 안으로 설명한다. 그러...,표준 모형,Aug-0-00024,"{'answer_start': [0], 'text': ['표준 모형']}",100024,10024


In [17]:
restriction_generation_data = {'question': [],
                   'context': [],
                   'title': [],
                   'id': [],
                   'answers': [],
                   'document_id': [],
                   '__index_level_0__': []}
# __index_level_0__: 10001부터 번호 Numbering\n",
# document_id: 100001부터 Numbering\n",
# answers: 알맞게 삽입\n",
# id: Aug-0-00001부터 Nubering\n",
error_list = []
for i in tqdm(range(len(wiki_qa_df))) :
    text = wiki_qa_df['text'][i]
    title = wiki_qa_df['title'][i]
    try :
        question = qg(title, text)
    except :
        error_list.append(i)
        continue
    answer_start = text.index(title)
    answers = {'answer_start': [answer_start], 'text': [title]}
    
    query_cls_hidden = get_cls_token(question)
    passage_cls_hidden = get_cls_token(text)
    similarity = cosine_similarity(query_cls_hidden, passage_cls_hidden)

    if similarity[0][0] >= 0.6 and title not in question :
        __index_level_0__ = 10001 + i
        document_id = 100001 + i
        id_number = 1 + i
        id_number = str(id_number)
        id_ = f'Aug-0-{id_number.zfill(5)}'
        
        restriction_generation_data['title'].append(title)
        restriction_generation_data['context'].append(text)
        restriction_generation_data['question'].append(question)
        restriction_generation_data['id'].append(id_)
        restriction_generation_data['answers'].append(answers)
        restriction_generation_data['document_id'].append(document_id)
        restriction_generation_data['__index_level_0__'].append(__index_level_0__)


  0%|          | 31/28765 [00:30<7:43:30,  1.03it/s][nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
  0%|          | 79/28765 [01:09<5:58:10,  1.33it/s][nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
  0%|          | 104/28765 [01:28<6:24:09,  1.24it/s][nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
  0%|          | 113/28765 [01:33<7:41:43,  1.03it/s][nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
  0%|          | 126/28765 [01:42<6:58:50,  1.14it/s][nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
  1%|          | 145/28765 [01:54<6:01:22,  1.32it/s][n

In [18]:
restriction_generation_data = pd.DataFrame(restriction_generation_data)
restriction_generation_data.to_csv('../data/train_dataset/aug_train_dataset_sub.csv', index = False)

## inference 한 번 하기!